# Import Required Libraraies

In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# Read Data

In [2]:
train_data = pd.read_csv('/kaggle/input/titanic/train.csv')
test_data = pd.read_csv('/kaggle/input/titanic/test.csv')
gender_sub = pd.read_csv('/kaggle/input/titanic/gender_submission.csv')

In [3]:
test_data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


# Data Purification

## Name
### Extract titles from names

In [4]:
import re
train_data['Title'] = train_data['Name'].apply(lambda x: re.search(' ([A-Za-z]+)\.', x).group(1))
test_data['Title'] = test_data['Name'].apply(lambda x: re.search(' ([A-Za-z]+)\.', x).group(1))

# Display the unique titles
print(f'All unique titles in names (Train): {train_data["Title"].unique()}')
print(f'All unique titles in names (Test): {test_data["Title"].unique()}')

All unique titles in names (Train): ['Mr' 'Mrs' 'Miss' 'Master' 'Don' 'Rev' 'Dr' 'Mme' 'Ms' 'Major' 'Lady'
 'Sir' 'Mlle' 'Col' 'Capt' 'Countess' 'Jonkheer']
All unique titles in names (Test): ['Mr' 'Mrs' 'Miss' 'Master' 'Ms' 'Col' 'Rev' 'Dr' 'Dona']


In [5]:
# Train Data
train_data['Title'] = train_data['Title'].replace(['Lady', 'Countess', 'Capt', 'Col', 'Don', 'Dr','Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Other')
train_data['Title'] = train_data['Title'].replace(['Mlle', 'Ms'], 'Miss')
train_data['Title'] = train_data['Title'].replace('Mme', 'Mrs')

# Test Data
test_data['Title'] = test_data['Title'].replace(['Lady', 'Countess', 'Capt', 'Col', 'Don', 'Dr','Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Other')
test_data['Title'] = test_data['Title'].replace(['Mlle', 'Ms'], 'Miss')
test_data['Title'] = test_data['Title'].replace('Mme', 'Mrs')

In [6]:
print(f'Train Titles: {train_data["Title"].unique()}')
print(f'Test Titles: {test_data["Title"].unique()}')

Train Titles: ['Mr' 'Mrs' 'Miss' 'Master' 'Other']
Test Titles: ['Mr' 'Mrs' 'Miss' 'Master' 'Other']


### Drop Names

In [7]:
train_data = train_data.drop('Name', axis=1)
test_data = test_data.drop('Name', axis=1)

## Ticket
### Drop ticket columns

In [8]:
# Drop Ticket Column
train_data = train_data.drop('Ticket', axis=1)
test_data = test_data.drop('Ticket', axis=1)

In [9]:
train_data.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,Title
0,1,0,3,male,22.0,1,0,7.2500,NaN,S,Mr
1,2,1,1,female,38.0,1,0,71.2833,C85,C,Mrs
2,3,1,3,female,26.0,0,0,7.9250,NaN,S,Miss
3,4,1,1,female,35.0,1,0,53.1000,C123,S,Mrs
4,5,0,3,male,35.0,0,0,8.0500,NaN,S,Mr


In [10]:
test_data.head()

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,Title
0,892,3,male,34.5,0,0,7.8292,NaN,Q,Mr
1,893,3,female,47.0,1,0,7.0000,NaN,S,Mrs
2,894,2,male,62.0,0,0,9.6875,NaN,Q,Mr
3,895,3,male,27.0,0,0,8.6625,NaN,S,Mr
4,896,3,female,22.0,1,1,12.2875,NaN,S,Mrs
